In [1]:
import logging
import shutil
import numpy as np
from ConfigSpace.hyperparameters import UniformFloatHyperparameter
from smac.configspace import ConfigurationSpace
from smac.facade.smac_bo_facade import SMAC4BO
from smac.scenario.scenario import Scenario

In [2]:
logging.basicConfig(level=logging.INFO) 

In [1]:
from models.utils import Log

In [ ]:
Log()

In [9]:
import deminf_data
f = deminf_data.Objective.from_name('1_Bot_4_Sim', negate=True, type_of_transform='logarithm')

[ -6.90775528  -6.90775528 -34.53877639 -34.53877639]
[4.60517019 4.60517019 1.60943791 1.60943791]


In [1]:
from models.SMAC import SMAC

In [2]:
model = SMAC(5, '1_Bot_4_Sim', verbose=True)

In [3]:
X, Y = model.run()

[-1.1512925464970225, -1.1512925464970225, -16.46466924123829, -16.46466924123829]
1
[-6.907407134415176, 4.538484222597141, -33.949950971042874, -34.53741158256178]
1
[-1.1124202842406694, -1.165540036460298, -16.468878495881597, -16.284410739026765]
1
[-0.9862129619133642, -1.2251977684676856, -16.501772119504743, -15.051940338714275]
1
[-0.9272280833889086, -1.2332048482338056, -16.52455379183647, -14.514663835176147]
1


In [4]:
print(model.f)

In [5]:
model.f.count

0

In [5]:
X, Y

([], [])

In [3]:
def rosenbrock_2d(x):
    """ The 2 dimensional Rosenbrock function as a toy model
    The Rosenbrock function is well know in the optimization community and
    often serves as a toy problem. It can be defined for arbitrary
    dimensions. The minimium is always at x_i = 1 with a function value of
    zero. All input parameters are continuous. The search domain for
    all x's is the interval [-5, 10].
    """
    x1 = x["x0"]
    x2 = x["x1"]

    val = 100. * (x2 - x1 ** 2.) ** 2. + (1 - x1) ** 2.
    return val

In [4]:
cs = ConfigurationSpace()
x0 = UniformFloatHyperparameter("x0", -5, 10, default_value=-3)
x1 = UniformFloatHyperparameter("x1", -5, 10, default_value=-4)
cs.add_hyperparameters([x0, x1])

# Scenario object
scenario = Scenario({"run_obj": "quality",  # we optimize quality (alternatively runtime)
                     "runcount-limit": 10,  # max. number of function evaluations; for this example set to a low number
                     "cs": cs,  # configuration space
                     "deterministic": "true"
                     })

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = rosenbrock_2d(cs.get_default_configuration())
print("Default Value: %.2f" % def_value)

# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC4BO(scenario=scenario,
               rng=np.random.RandomState(42),
               tae_runner=rosenbrock_2d,
               )

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2021-05-30_18:56:10_593545
INFO:smac.facade.smac_bo_facade.SMAC4BO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 2 configurations
INFO:smac.facade.smac_bo_facade.SMAC4BO:<class 'smac.facade.smac_bo_facade.SMAC4BO'>


Default Value: 16916.00
Optimizing! Depending on your machine, this might take a few minutes.


In [9]:
smac.optimize()

INFO:smac.optimizer.smbo.SMBO:State Restored! Starting optimization with incumbent Configuration:
  x0, Value: -0.7845334063352647
  x1, Value: 0.35959503070043475

INFO:smac.optimizer.smbo.SMBO:State restored with following budget:
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 3
INFO:smac.stats.stats.Stats:#Submitted target algorithm runs: 10 / 10.0
INFO:smac.stats.stats.Stats:#Finished target algorithm runs: 10 / 10.0
INFO:smac.stats.stats.Stats:#Configurations: 10
INFO:smac.stats.stats.Stats:Used wallclock time: 15.17 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 0.07 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent change

Configuration:
  x0, Value: -0.7845334063352647
  x1, Value: 0.35959503070043475

In [12]:
shutil.rmtree(smac.output_dir.split('/')[0])

In [27]:
cs_output['x1']

0.35959503070043475

In [10]:
x0 = -0.7845334063352647
x1 = 0.35959503070043475
x2 = rosenbrock_2d({'x0': x0, 'x1': x1})

In [22]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
X, Y = np.linspace(-5, 10, 10), np.linspace(-5, 10, 10)
X, Y = np.meshgrid(X, Y)
Z = np.array([rosenbrock_2d({'x0': x, 'x1': y}) for x, y in zip(X, Y)])
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])
fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.add_trace(go.Scatter3d(x=[x0], y=[x1], z=[x2]))
fig.show()